### Installing packages

In [1]:
# Installing packages
#!pip3 install pandas
#!pip3 install biopython

# Importing packages
import Bio
import math
import os
import sys
import argparse
import shutil
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Data import CodonTable
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from itertools import chain
from tqdm import tqdm
#from google.colab import files   # For working with colab only
#from google.colab import drive   # For working with colab only

# Asserting email for Entrez
Entrez.email = "bogdan.sotnikov.1999@mail.ru"

# Google drive mounting
#drive.mount('/content/gdrive')    # For working with colab only

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument("path", type=str)
parser.add_argument("mail", type=str)
parser.add_argument("threads", type=int)

arguments = parser.parse_args()

json_file_name = arguments.path
Entrez.email = arguments.mail
threads = arguments.threads
# Если аргумент есть в аргументах, сделать api

In [ ]:
json_path = os.path.join("../data/jsons", json_file_name)
with open(json_path, "r") as json_organism:
    json_organism = json.load(json_organism)


organism_name = json_organism[0]
complete_ids = list(json_organism[1].values())[0]

In [20]:
if type(complete_ids[len(complete_ids)-1]) == list:
    scaffs = complete_ids[len(complete_ids)-1]
    del complete_ids[len(complete_ids)-1]
    complete_ids += scaffs

In [78]:
# Code from source above, with modifications. It is for choosing GenBank nucleotide data only.
def extract_insdc(links):
    linkset = [ls for ls in links[0]['LinkSetDb'] if
              ls['LinkName'] == 'assembly_nuccore_insdc']
    if 0 != len(linkset):
        uids = [link['Id'] for link in linkset[0]['Link']]
    else:
        uids = 0
    return uids

In [23]:
# Creating a variables for futher work with links
#organism = organism_name
organism = organism_name
db_search = "assembly"
db_current = "nucleotide"

In [6]:
# # Getting summary about links
# idlist = search_record["IdList"]
# complete_ids = []   # List of full completed genomes' ids
# for ids in tqdm(idlist):
#     handle = Entrez.esummary(db=db_search, id=ids)
#     record = Entrez.read(handle)
#     if record['DocumentSummarySet']['DocumentSummary'][0]['AssemblyStatus'] == "Complete Genome":
#         complete_ids.append(ids)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [06:04<00:00,  1.80s/it]


In [92]:
organism = "Acinetobacter_baumannii"
complete_ids = ["12814861", "12262961", "836081", "7051501", "7867071", "648281", "7332281", "2799101", "2228081", "836121", "13538571", "12263621", "590291", "935061", "12718751", "1629471", "648301", "1950371", "1115631", "13199171", "1032731", "648271", "12719911", "10283131", "1064241", "8588321", "7390001", "13538641", "11455031", "15866881", "9392531", "6719611", "12159421", "2552021", "12812961", "791871", "13538581", "9928511", "836141", "12718671", "2650481", "8237601", "1950351", "12266941", "12720111", "15073091", "7061401", "16400971", "13279381", "13357651", "7365931", "12719961", "13199731", "12719951", "15866931", "12718661", "7332291", "12263751", "1141461", "12208811", "12718651", "1947011", "1434611", "894901", "11454961", "10577061", "1991081", "2819621", "7051381", "13538671", "12813031", "16296081", "5131601", "12718731", "12267161", "12720041", "13538551", "12718641", "12814821", "11454981", "12267521", "214351", "1064221", "2161061", "13982451", "11454971", "248141", "13538541", "618731", "13544441", "836091", "12267261", "14216471", "14310881", "12267071", "8147131", "755881", "2112931", "13195821", "9714881"]

In [93]:
# Taking ids for fetching. It collected all non-duplicated links in nucleotide database from assembly database.
links = []
links_checked = []
n = 0
for complete_id in tqdm(complete_ids):
    link_handle = Entrez.elink(dbfrom=db_search, db=db_current, from_uid=complete_id)
    link_record = Entrez.read(link_handle)
    print(link_record)
    uids = extract_insdc(link_record)
    if uids != 0:
        for uid in uids:
            print(uid)
            if uid not in links_checked:  # Checking for duplicates
                links_checked.append(uid)
                links.append((uid, n))
                cumulative = 1
            else:
                cumulative = 0
        n += cumulative

  1%|█                                                                                                       | 1/100 [00:01<01:59,  1.21s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2245020635'}, {'Id': '2245020632'}, {'Id': '2245020610'}, {'Id': '2242516291'}, {'Id': '2242516202'}, {'Id': '2242512623'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2242516291'}, {'Id': '2242516202'}, {'Id': '2242512623'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2245020635'}, {'Id': '2245020632'}, {'Id': '2245020610'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['12814861']}]
2242516291
2242516202
2242512623


  2%|██                                                                                                      | 2/100 [00:02<01:53,  1.16s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2203669227'}, {'Id': '2203085011'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2203085011'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2203669227'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['12262961']}]
2203085011


  3%|███                                                                                                     | 3/100 [00:03<01:41,  1.05s/it]

...

[{'LinkSetDb': [{'Link': [{'Id': '1520152424'}, {'Id': '1520152423'}, {'Id': '1520152422'}, {'Id': '1520152421'}, {'Id': '1518936647'}, {'Id': '1518936303'}, {'Id': '1518936279'}, {'Id': '1518936265'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '1518936647'}, {'Id': '1518936303'}, {'Id': '1518936279'}, {'Id': '1518936265'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '1520152424'}, {'Id': '1520152423'}, {'Id': '1520152422'}, {'Id': '1520152421'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['2112931']}]
1518936647
1518936303
1518936279
1518936265


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [01:47<00:01,  1.00s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2264259596'}, {'Id': '2263642300'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2263642300'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2264259596'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['13195821']}]
2263642300


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:48<00:00,  1.09s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2022135500'}, {'Id': '2022135498'}, {'Id': '2022135497'}, {'Id': '2022135488'}, {'Id': '2022135401'}, {'Id': '2021493314'}, {'Id': '2021493302'}, {'Id': '2021493219'}, {'Id': '2021493015'}, {'Id': '2021489251'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2021493314'}, {'Id': '2021493302'}, {'Id': '2021493219'}, {'Id': '2021493015'}, {'Id': '2021489251'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2022135500'}, {'Id': '2022135498'}, {'Id': '2022135497'}, {'Id': '2022135488'}, {'Id': '2022135401'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['9714881']}]
2021493314
2021493302
2021493219
2021493015
2021489251


In [95]:
links

[('2242516291', 0),
 ('2242516202', 0),
 ('2242512623', 0),
 ('2203085011', 1),
 ('1087730795', 2),
 ('1087730686', 2),
 ('1836070421', 3),
 ('1836070410', 3),
 ('1836066657', 3),
 ('1877588604', 4),
 ('1877588482', 4),
 ('1877585046', 4),
 ('1000993161', 5),
 ('1866706066', 6),
 ('1866702431', 6),
 ('1635522416', 7),
 ('1635522322', 7),
 ('1635518656', 7),
 ('1566474420', 8),
 ('1566474290', 8),
 ('1566470628', 8),
 ('1087746352', 9),
 ('1087746255', 9),
 ('2289938025', 10),
 ('2289938024', 10),
 ('2289938023', 10),
 ('2289938022', 10),
 ('2203034173', 11),
 ('902756494', 12),
 ('1125869058', 13),
 ('1125868940', 13),
 ('1125865289', 13),
 ('2238627894', 14),
 ('2238627801', 14),
 ('2238624079', 14),
 ('1359185821', 15),
 ('1359181956', 15),
 ('1001000371', 16),
 ('1474724526', 17),
 ('1474720871', 17),
 ('1199168558', 18),
 ('1199168556', 18),
 ('2263959133', 19),
 ('1051000319', 20),
 ('1050996618', 20),
 ('1000989619', 21),
 ('2238639514', 22),
 ('2238639494', 22),
 ('2238635766', 

In [97]:
# Collecting data about assemblies
gb_records = []
for link in tqdm(links):
    print(link)
    gb_handle = Entrez.efetch(db=db_current, rettype="gb", retmode="text", id=link[0])
    gb_record = SeqIO.read(gb_handle, 'genbank')
    gb_records.append((gb_record, link[1]))

  0%|                                                                                                                | 0/241 [00:00<?, ?it/s]

('2242516291', 0)


  0%|▍                                                                                                     | 1/241 [00:29<1:59:50, 29.96s/it]

('2242516202', 0)


  1%|▊                                                                                                     | 2/241 [00:35<1:01:32, 15.45s/it]

...

('1877588604', 4)


  4%|████▎                                                                                                  | 10/241 [03:23<58:25, 15.17s/it]

('1877588482', 4)


  4%|████▏                                                                                                | 10/241 [04:23<1:41:22, 26.33s/it]


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [91]:
gb_records[1200][0]

SeqRecord(seq=Seq('TAAACGCAGCTCATCCTGAATGGATTCCTGCTGTTGTGCTCTACCAGGAAATTT...TCT'), id='PRDY01000125.1', name='PRDY01000125', description='Acinetobacter indicus strain IHIT31216 NODE_138_length_301_cov_139.983, whole genome shotgun sequence', dbxrefs=['BioProject:PRJNA432656', 'BioSample:SAMN08456626'])

In [86]:
!pwd

/home/asklepiad/bioinf/start_codons/BI_project_2022/C_psittaci_folder/scripts


On the next stage we need to create fasta files for each assembly, to reaanotate them.

In [1]:
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
print(name)

P_penneri


In [23]:
dna_type, tuples, source_list = [], [], [] # Creating list for identyfing the number of every assemblie DNA molecules (chromosome and any plasmids)
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
for rec in tqdm(gb_records):
    source = rec[1] # Number of assembly
    if "plasmid" in rec[0].description:
        dna_type.append("plasmid")
    else:
        dna_type.append("chromosome")
    source_list.append(source) 
    number = source_list.count(source) # Counting the DNA molecule of assembly
    tuples.append((source, number))
    mask = f"{name}{source}_{number}"
    with open (f"../{name}/data/for_prokka_fasta/{mask}.fasta", "w") as for_prokka_fasta:
        for_prokka_fasta.write(">")
        for_prokka_fasta.write(mask)
        for_prokka_fasta.write("\n")
        for_prokka_fasta.write(str(rec[0].seq))
        for_prokka_fasta.write("\n")  

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 633.51it/s]


In [35]:
# Saving plasmid_code
jsonpc1 = json.dumps(tuples)
with open(f"../{name}/data/{name}_plasmid_code1.json", "w") as jspc1:
    jspc1.write(jsonpc1)
jsonpc2 = json.dumps(dna_type)
with open(f"../{name}/data/{name}_plasmid_code2.json", "w") as jspc2:
    jspc2.write(jsonpc2)